<a href="https://colab.research.google.com/github/soohyunme/exercise/blob/main/Code/02_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
cd /content/drive/MyDrive/4조/01.체육공모전/Code

/content/drive/.shortcut-targets-by-id/159Rw9eptrdiP8wihxnfPLcnRcwjx1x8n/4조/01.체육공모전/Code


# 모듈 로드

In [25]:
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# 데이터 로드

In [26]:
df_origin = pd.read_csv('../Data/04.processed_data.csv',encoding='cp949')
group = pd.read_csv('../Data/user_group.csv',encoding='cp949')
ex_db = pd.read_csv('../Data/exercise_DB.csv',encoding='cp949')
ex_db['NEW_EX_NM'] = ex_db['EX_NM'].apply(lambda x : x.replace(' ',''))
ex_dict = dict(zip(list(ex_db['NEW_EX_NM']),list(ex_db['EX_NM'])))


In [27]:
user_db = df_origin[:-20]
sample_user = df_origin[-20:].iloc[:,:-3]

# 함수 정의

## 데이터프레임 변환 함수

In [28]:
# 나이를 연령대로 변환
def age_group(df):
  df['AGRDE_SE'] = df['TEST_AGE'].apply(lambda x : '10대' if x<20 else (\
                                                    '20대' if x<30 else (\
                                                      '30대' if x<40 else (\
                                                    '40대' if x<50 else (\
                                                    '50대' if x<60 else (\
                                                      '60대' if x<70 else (\
                                                    '70대 이상' )))))))
  return df

In [29]:
# 키와 몸무게를 통해 BMI 계산, BMI를 통해 비만 단계 변환
def bmi_group(df):
  bmi = pd.Series(round(df['WEIGHT'] / (df['HEIGHT']/100)**2,1),name='BMI')
  bmi_g = bmi.apply(lambda x : '3단계비만' if x>=35 else (\
                                                    '2단계비만' if x>=30 else (\
                                                      '1단계비만' if x>=25 else (\
                                                    '비만전단계' if x>=23 else (\
                                                    '정상' if x>=18.5 else '저체중')))))
  result = pd.concat([df,bmi_g],axis=1)
  return result

In [30]:
# Add user group column
def add_group(df):
  result = df.merge(group,how='inner')
  return result

In [31]:
def transform(df):
  result = age_group(df.copy())
  result = bmi_group(result)
  result = add_group(result)
  if '준비운동' in df.columns:
    result = result[['USER_ID', 'GROUP_ID', '준비운동', '본운동', '마무리운동']]
  else:
    result = result[['USER_ID', 'GROUP_ID']]
  return result

# 데이터셋 변환  
연령대, 운동강도, 성별, BMI 등급을 통한 그룹 변환

In [32]:
# User DB & sample user data transform
trans_db = transform(user_db)
trans_sample = transform(sample_user)

# exercise set df
set_df = pd.DataFrame(trans_db.drop('USER_ID',axis=1).value_counts()).reset_index().rename(columns={0:'COUNT'})

## 최빈 운동 세트 선정 함수

In [33]:
# input : exercise list / output : sparse matrix
def sparse_matrix(ex_list):
  te = TransactionEncoder()
  te_ary = te.fit(ex_list).transform(ex_list)
  sparse_matrix = pd.DataFrame(te_ary, columns=te.columns_)
  return sparse_matrix

In [34]:
def ranking(pre_df, main_df, finish_df, ex_list):
  df_list = [pre_df, main_df, finish_df] # 단계별 연관분석 결과값
  result_list = [] # 결과값을 리스트에 담아 하나씩 출력
  result_df = pd.DataFrame(columns=pre_df.columns) # 빈 DataFrame 생성
  
  for i in range(len(df_list)):
    tmp_df = df_list[i] # 운동 단계별 for문 pre, main, finish
    
    for each_ex in ex_list[i].split(','): # 각 단계별 운동들을 리스트로 분할
      each_ex = each_ex.replace(' ','') # 운동이름에 공백 제거
      each_result = tmp_df[tmp_df['antecedents']==frozenset({each_ex})].sort_values('support') # 해당하는 운동을 연관분석을 통해 지지도 순으로 정렬
      each_result['consequents'] = each_result['consequents'].apply(lambda x : list(x)[0]) # frozenset to string
      length = 3 if len(each_result) else len(each_result) # set length
      result_df = pd.concat([result_df,each_result.iloc[:length]]) # 연관성이 높은 운동이 3개 이상일 경우 3개만 저장
    result_list.append(result_df['consequents']) # 결과값 리스트에 추가
    result_df = None # 초기화
  return list(result_list[0]), list(result_list[1]), list(result_list[2]) # 준비, 본, 마무리 운동 연관성 높은 운동 리턴 / type : Series

In [101]:
# select top 1 exercise set from user info
# input : group_id / ouput : top 1 exercise set(pre, main, finish / Series)
def mode_set(g_id, min_support = 0.01, min_threshold=0.2):
  global max_ex_set
  user_set = set_df[set_df['GROUP_ID']==g_id] # 유저가 해당하는 그룹의 처방 운동 세트 빈도
  max_ex_set = list(user_set[user_set['COUNT'] == user_set['COUNT'].max()].iloc[0,1:4]) # 가장 많이 처방된 운동 세트

  # 해당 그룹의 단계별 운동 리스트 생성
  pre_ex_all = list(user_set['준비운동'].apply(lambda x : x.replace(' ','')).apply(lambda x : x.split(','))) 
  main_ex_all = list(user_set['본운동'].apply(lambda x : x.replace(' ','')).apply(lambda x : x.split(',')))
  finish_ex_all = list(user_set['마무리운동'].apply(lambda x : x.replace(' ','')).apply(lambda x : x.split(',')))

  # 단계별 sparse matrix 생성
  pre_sparse = sparse_matrix(pre_ex_all)
  main_sparse = sparse_matrix(main_ex_all)
  finish_sparse = sparse_matrix(finish_ex_all)

  # 지지도가 0.01 이상인 운동 선택
  pre_itemsets = apriori(pre_sparse, min_support = min_support, use_colnames=True)
  main_itemsets = apriori(main_sparse, min_support = min_support, use_colnames=True)
  finish_itemsets = apriori(finish_sparse, min_support = min_support, use_colnames=True)

  # 그 중에서 신뢰도가 0.5 이상인 운동 선택
  pre_select = association_rules(pre_itemsets, metric="confidence", min_threshold=min_threshold)
  main_select = association_rules(main_itemsets, metric="confidence", min_threshold=min_threshold)
  finish_select = association_rules(finish_itemsets, metric="confidence", min_threshold=min_threshold)
  
  # consequents의 길이가 1인 것만 선택 -> 1개의 운동만 추천하도록
  pre_select = pre_select[pre_select['consequents'].apply(lambda x : len(x) == 1)]
  main_select = main_select[main_select['consequents'].apply(lambda x : len(x) == 1)] 
  finish_select = finish_select[finish_select['consequents'].apply(lambda x : len(x) == 1)]
  
  # ranking 
  pre_rank, main_rank, finish_rank = ranking(pre_select, main_select, finish_select, max_ex_set)

  # select exercise 
  select_exercise(pre_rank, main_rank, finish_rank, max_ex_set)

  return pre_rank, main_rank, finish_rank

mode_set(32)

[['하지루틴스트레칭', '상지루틴스트레칭'], ['전신루틴스트레칭'], ['목스트레칭', '등/어깨뒤쪽스트레칭', '어깨뒤쪽스트레칭', '양팔벌려전신비틀기', '골반스트레칭'], ['전신루틴스트레칭'], ['전신루틴스트레칭'], ['깍지끼고상체숙이기', '아래팔스트레칭', '넙다리스트레칭', '팔꿈치맞닿아원그리기'], ['하지루틴스트레칭', '상지루틴스트레칭', '전신루틴스트레칭'], ['전신루틴스트레칭', '유산소운동전동적루틴스트레칭'], ['아래팔스트레칭', '엉덩이스트레칭', '넙다리스트레칭', '허리스트레칭', '배스트레칭', '옆구리스트레칭', '어깨스트레칭'], ['걷기'], ['하지루틴스트레칭', '상지루틴스트레칭', '전신루틴스트레칭', '유산소운동전동적루틴스트레칭'], ['하지루틴스트레칭', '상지루틴스트레칭'], ['하지루틴스트레칭', '상지루틴스트레칭'], ['목스트레칭', '등/어깨뒤쪽스트레칭', '가슴/어깨앞쪽스트레칭', '아래팔스트레칭'], ['자전거타기', '걷기'], ['서서뒤꿈치들기', '유산소운동전동적루틴스트레칭'], ['앉았다일어서기', '무릎높여제자리달리기', '팔벌려뛰기', '옆으로펀치하기'], ['걷기', '유산소운동전동적루틴스트레칭'], ['하지루틴스트레칭', '상지루틴스트레칭', '유산소운동전동적루틴스트레칭'], ['하지루틴스트레칭', '상지루틴스트레칭', '전신루틴스트레칭'], ['전신루틴스트레칭', '유산소운동전동적루틴스트레칭', '짐볼을이용한동적루틴스트레칭', '하지루틴스트레칭'], ['정적스트레칭루틴프로그램'], ['전신루틴스트레칭'], ['하지루틴스트레칭', '상지루틴스트레칭'], ['걷기'], ['걷기'], ['걷기', '자전거타기', '스텝박스'], ['유산소운동전동적루틴스트레칭', '자전거타기', '걷기'], ['전신루틴스트레칭'], ['유산소운동전동적루틴스트레칭', '좌식생활자를위한동적루틴스트레칭'], ['자가근막이완술루틴스트레칭', '목스트레칭', '상지루틴스트레칭', '하지루틴스트레칭'], ['자

In [42]:
import random
random_num = random.randint(1, 100) 

In [94]:
a,b,c = mode_set(32)

In [98]:
# select main routine
def select_exercise(pre_list, main_list, finish_list, max):
  main_reco_list = []
  sub_reco_list = []
  tmp_list = []
  aa = pre_list

  random_num = random.randint(1, 100) 
  # Main recommend set
  for str in max:
    ex_list = str.split(',')
    main_reco_list.append(ex_list[random_num %len(ex_list)])
  print(aa)
  aa.remove(main_reco_list[0].replace(' ',''))
  print(aa)
  return main_reco_list, sub_reco_list
main_reco, sub_reco = select_exercise(a,b,c,max_ex_set)

['유산소운동전동적루틴스트레칭', '전신루틴스트레칭', '상지루틴스트레칭', '하지루틴스트레칭']
상지 루틴 스트레칭
['유산소운동전동적루틴스트레칭', '전신루틴스트레칭', '하지루틴스트레칭']


# 맞춤 운동 추천

In [46]:
a, b, c = mode_set(91)

In [ ]:
b

In [ ]:
for u_id, g_id in trans_sample.values:
  #recommend_list = list(mode_set(g_id))
  #sparse_matrix(g_id, recommend_list)
  print(list(mode_set(g_id))) # 준비, 본, 마무리

  break